In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip install -U skorch

     |████████████████████████████████| 112kB 41.9MB/s 


In [0]:
from skorch import NeuralNetRegressor, NeuralNetClassifier, NeuralNet
import skorch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as ttsplit
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split as ttsplit
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor
from skorch.dataset import CVSplit


from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, BaggingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import cat
import torch.optim as optim
from torch.utils import data as data_torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os


In [0]:
# our = 0
# data_fn = 'drive/My Drive/CS 598PS Final Project/data/our_data.csv' if our else 'drive/My Drive/CS 598PS Final Project/data/best_score_data.csv'

data_fn = 'drive/My Drive/CS 598PS Final Project/data/combined_data_smarter_2.csv'

data_wTarget = pd.read_csv(data_fn, header=0, index_col=0)
data = data_wTarget[[col for col in data_wTarget.columns if col != 'Yards']]
target = data_wTarget['Yards']
data_np = data.to_numpy(dtype='float')
target_np = target.to_numpy(dtype='int')


In [0]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split as ttsplit

X_train, X_test, Y_train, Y_test = ttsplit(data_np, target_np, test_size=0.2)
# X_train, X_val, Y_train, Y_val = ttsplit(X_train, Y_train, test_size=0.2)

#scaler = MinMaxScaler()
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# X_val = scaler.transform(X_val)

In [0]:
# from sklearn.decomposition import PCA
# pca_comps = 125 if our else 20
# pca = PCA(n_components=pca_comps)
# pca.fit(X_train)
# np.sum(pca.explained_variance_ratio_)

# # X_train_pca = pca.transform(X_train)
# # X_test_pca = pca.transform(X_test)
# # X_val_pca = pca.transform(X_val)
# X_train_pca = X_train
# X_test_pca = X_test
# X_val_pca = X_val

# print(X_train_pca.shape, X_test_pca.shape)

In [0]:
def get_one_hot(Y):
  res = np.zeros((Y.shape[0], 199))
  for i in range(Y.shape[0]):
    res[i][Y[i]+99] = 1
  return res 

def get_pred(Y_one_hot):
  res = np.zeros((Y_one_hot.shape[0],))
  for i in range(Y_one_hot.shape[0]):
    res[i] = np.argmax(Y_one_hot[i])-99
  return res

Y_train_one_hot = get_one_hot(Y_train)
Y_test_one_hot = get_one_hot(Y_test)
print(Y_train_one_hot.shape)

(18536, 199)


In [0]:
def calc_crps(Y, Y_pred):
  y_true = np.clip(np.cumsum(Y, axis=1), 0, 1)
  y_pred = np.clip(np.cumsum(Y_pred, axis=1), 0, 1)
  val_s = ((y_true - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * Y.shape[0])
  val_s = np.round(val_s, 6)
  return val_s

In [0]:
def pred_probs_to_199_classes(probs, curr_classes):
  y = np.zeros((probs.shape[0], 199))
  for i in range(probs.shape[0]):
    for class_idx, class_prob in enumerate(list(probs[i])):
        y[i][99+curr_classes[class_idx]] = class_prob
  return y

In [0]:
class NFLRegressorNet3(nn.Module):
    def __init__(self):
        super(NFLRegressorNet3, self).__init__()
        
        Nsamples, Nfeatures = X_train.shape
        print("Nfeatures: ", Nfeatures)
        self.net = nn.Sequential(

            nn.Linear(Nfeatures, 1024),
            nn.ReLU(1024),
            nn.Dropout(0.5),
            nn.BatchNorm1d(1024),

            nn.Linear(1024, 512),
            nn.ReLU(512),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),

            nn.Linear(512, 256),
            nn.ReLU(256),
            nn.Dropout(0.5),
            nn.BatchNorm1d(256),

            nn.Linear(256, 199),
            nn.ReLU(199),
            nn.Dropout(0.5),
            nn.BatchNorm1d(199),

            # nn.Softmax(dim=-1),
            # nn.Linear(199, 1)
        )
       
    def forward(self, x, **kwargs):
        batch_size = x.shape[0]
        x = x.float()
        # x = x.view(batch_size, -1)
        out = self.net(x)

        return out

# net = NeuralNet(
#     NFLRegressorNet3,
#     max_epochs=100,
#     train_split=CVSplit(10, stratified=True),
#     lr=0.1,
#     optimizer=torch.optim.SGD,
#     optimizer__momentum=0.95,
#     iterator_train__shuffle=True,
#     criterion=torch.nn.CrossEntropyLoss,
#     callbacks=[skorch.callbacks.LRScheduler(policy='ReduceLROnPlateau', monitor='valid_loss')]
# )


In [0]:
from scipy.special import softmax

def eval_crps(net, X, y):
    y_pred = net.predict_proba(X)
    # print("y_pred shape before softmax:", y_pred.shape)
    # print("y_pred before softmax:", y_pred)
    y_pred = softmax(y_pred, axis=1)
    # print("y_pred shape after softmax:", y_pred.shape)
    # print("y_pred after softmax:", y_pred)
    # print("y shape:", y.shape)
    # print("y:", y)
    # print("y_pred sum along axis 1:", np.sum(y_pred, axis=1))
    loss = calc_crps(Y_to_199_one_hot(y), y_pred)
    return loss

def Y_to_199_one_hot(Y):
  y = np.zeros((Y.shape[0], 199))
  for idx, target in enumerate(list(Y)):
      y[idx][target] = 1
  return y

def Y_to_shiftd_199_one_hot(Y):
  y = np.zeros((Y.shape[0], 199))
  for idx, target in enumerate(list(Y)):
      y[idx][target+99] = 1
  return y

class NFLClassifierNet(nn.Module):
    def __init__(self, Nfeatures):
        super(NFLClassifierNet, self).__init__()
        
        # print("Nfeatures: {}".format(Nfeatures))

        self.net = nn.Sequential(
    
            nn.Linear(Nfeatures, 1024),
            nn.ReLU(1024),
            nn.Dropout(0.5),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 512),
            nn.ReLU(512),
            nn.Dropout(0.5),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(256),
            nn.Dropout(0.5),
            nn.BatchNorm1d(256),
            nn.Linear(256, 199),
            nn.ReLU(199),
            nn.Dropout(0.5),
            nn.BatchNorm1d(199),
            # nn.Softmax(1)
        )

        self.classes_ = range(0,199)

       
    def forward(self, x):
      
        batch_size = x.shape[0]
        x = x.float()
        # x = x.view(batch_size, -1)
        out = self.net(x) # 512
        return out

In [0]:
# Bagging method

# Define classifiers 
et = ExtraTreesClassifier(10)
rf = RandomForestClassifier(10)
dtc = DecisionTreeClassifier()
knn = KNeighborsClassifier(10)
svc = SVC(gamma='auto') #
rg = RidgeClassifier() # 
gnb = GaussianNB() #

classifier_list = [rf, et, dtc, knn, rg]

for clf in classifier_list:
  # print(clf)
  for max_samples in [0.4, 0.5, 0.6, 0.9]:
    bagging_clf = BaggingClassifier(base_estimator=clf,
                                    n_estimators=10,
                                    max_samples=max_samples,
                                    bootstrap=True,
                                    random_state=0)
    # bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=100, bootstrap=True)
    bagging_clf= bagging_clf.fit(X_train, Y_train)
    bagging_score = bagging_clf.predict_proba(X_test)
    print("[{} Bagging] Score (w/ max_samples={}): {}".format(clf.__class__.__name__, max_samples,
                                          calc_crps(Y_test_one_hot,
                                                    pred_probs_to_199_classes(bagging_score, bagging_clf.classes_))))

  # bagging_scores = cross_val_score(bagging_clf, X_train_pca, Y_train, cv=5, n_jobs=-1)
  # bagging_score_mean, bagging_score_std = bagging_scores.mean(), bagging_scores.std()
  # print("{}: Mean: {} Std: {} ".format(clf.__class__.__name__, bagging_score_mean, bagging_score_std))


[RandomForestClassifier Bagging] Score (w/ max_samples=0.4): 0.013297
[RandomForestClassifier Bagging] Score (w/ max_samples=0.5): 0.013316
[RandomForestClassifier Bagging] Score (w/ max_samples=0.6): 0.013327
[RandomForestClassifier Bagging] Score (w/ max_samples=0.9): 0.013324
[ExtraTreesClassifier Bagging] Score (w/ max_samples=0.4): 0.013441
[ExtraTreesClassifier Bagging] Score (w/ max_samples=0.5): 0.013428
[ExtraTreesClassifier Bagging] Score (w/ max_samples=0.6): 0.013408
[ExtraTreesClassifier Bagging] Score (w/ max_samples=0.9): 0.013391
[DecisionTreeClassifier Bagging] Score (w/ max_samples=0.4): 0.014396
[DecisionTreeClassifier Bagging] Score (w/ max_samples=0.5): 0.01425
[DecisionTreeClassifier Bagging] Score (w/ max_samples=0.6): 0.01447
[DecisionTreeClassifier Bagging] Score (w/ max_samples=0.9): 0.014325
[KNeighborsClassifier Bagging] Score (w/ max_samples=0.4): 0.014005


KeyboardInterrupt: ignored

In [0]:
rf = RandomForestClassifier(10)

clf=rf
max_samples=0.6
bagging_clf = BaggingClassifier(base_estimator=clf,
                                n_estimators=100,
                                max_samples=max_samples,
                                bootstrap=True,
                                random_state=0)
# bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=100, bootstrap=True)
bagging_clf= bagging_clf.fit(X_train, Y_train)
bagging_score = bagging_clf.predict_proba(X_test)
print("[{} Bagging] Score (w/ max_samples={}): {}".format(clf.__class__.__name__, max_samples,
                                      calc_crps(Y_test_one_hot,
                                                pred_probs_to_199_classes(bagging_score, bagging_clf.classes_))))

[RandomForestClassifier Bagging] Score (w/ max_samples=0.6): 0.013123


In [0]:
clf = RandomForestClassifier(n_estimators=50, max_depth=10)
bagging_clf = BaggingClassifier(base_estimator=clf,
                                    n_estimators=25,
                                    max_samples=0.9,
                                    bootstrap=True,
                                    random_state=0)
# bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=100, bootstrap=True)
bagging_clf= bagging_clf.fit(X_train, Y_train)
bagging_score = bagging_clf.predict_proba(X_test)
print("[{} Bagging] Score (w/ max_samples={}): {}".format(clf.__class__.__name__, str(0.9),
                                      calc_crps(Y_test_one_hot,
                                                pred_probs_to_199_classes(bagging_score, bagging_clf.classes_))))

[RandomForestClassifier Bagging] Score (w/ max_samples=0.9): 0.013139


In [0]:
def sklearn_fit_predict(model, X_train, X_val, X_test, Y_train, Y_val, Y_test):

  model.fit(X_train, Y_train)
  Y_test_pred = model.predict(X_test)

  return Y_test_pred

from sklearn import metrics
def calc_regression_error(Y_test, Y_test_pred):
  errors = {
      'mean_absolute_error': metrics.mean_absolute_error(Y_test, Y_test_pred),
      'mean_squared_error': metrics.mean_squared_error(Y_test, Y_test_pred),
  }
  return errors

In [0]:
# Bagging method

# Define regressors 
from sklearn.tree import DecisionTreeRegressor
dtc = DecisionTreeRegressor(splitter='random')
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
from sklearn.linear_model import LinearRegression as LR
lr = LR()
from sklearn.svm import SVR
svr = SVR(gamma='auto')
from sklearn.neural_network import MLPRegressor
nnet = MLPRegressor(hidden_layer_sizes=(10,25,25,25,10), 
                  activation='relu', 
                  batch_size=400, 
                  learning_rate='adaptive', 
                  learning_rate_init=1e-4, 
                  max_iter=500,
                  solver='sgd',
                  momentum=0.5)

regressor_list = [dtc, knn, lr, svr]

for reg in regressor_list:
  # print(clf)
  bagging_reg = BaggingRegressor(base_estimator=reg,
                                  n_estimators=100,
                                  max_samples=0.4,
                                  bootstrap=True,
                                  random_state=0)
  # bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=100, bootstrap=True)
  test_predicts = sklearn_fit_predict(bagging_reg, X_train, X_val, X_test, Y_train, Y_val, Y_test)
  regression_errors = calc_regression_error(Y_test, test_predicts)
  print("[{} Bagging] Score: {}".format(reg.__class__.__name__,
                                        regression_errors))


[DecisionTreeRegressor Bagging] Score: {'mean_absolute_error': 3.7758942826321467, 'mean_squared_error': 37.48612450916936}


KeyboardInterrupt: ignored

In [0]:
# _, Nfeatures = X_train.shape
# net = NeuralNet(
#         NFLClassifierNet,
#         module__Nfeatures=Nfeatures,
#         max_epochs=50,
#         device='cuda',
#         warm_start=True,
#         train_split=CVSplit(5, stratified=False),
#         lr=0.1,
#         optimizer=torch.optim.SGD,
#         optimizer__momentum=0.95,
#         iterator_train__shuffle=True,
#         criterion=torch.nn.CrossEntropyLoss,
#         callbacks=[skorch.callbacks.LRScheduler(policy='ReduceLROnPlateau', monitor='valid_loss', mode='min'),
#                    skorch.callbacks.EpochScoring(scoring=eval_crps, use_caching=True, name='valid_crps')]
#     )
# net.classes_ = range(0,199)
# net.n_classes_ = 199

# bagging_clf = BaggingClassifier(base_estimator=net,
#                                   n_estimators=5,
#                                   max_samples=0.4,
#                                   bootstrap=True,
#                                   random_state=12345)

# # bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=100, bootstrap=True)
# bagging_clf= bagging_clf.fit(X_train, Y_train)
# bagging_clf.classes_ = range(0,199)
# bagging_clf.n_classes_ = len(bagging_clf.classes_)
# bagging_score = bagging_clf.predict_proba(X_test)
# print("[{} Bagging] Score: {}".format(net.__class__.__name__,
#                                       calc_crps(Y_test_one_hot,
#                                                 bagging_score)))

In [0]:
print("[{} Bagging] Score: {}".format(net.__class__.__name__,
                                      calc_crps(Y_test_one_hot,
                                                softmax(bagging_score, axis=1))))

[NeuralNet Bagging] Score: 0.417943


In [0]:
bagging_score

array([[-0.98167943, -0.73439755, -0.55721346, ..., -1.04335454,
        -1.04236326, -1.03617983],
       [-0.97983969, -0.79176793, -0.5129315 , ..., -1.0445835 ,
        -1.01660354, -1.03054804],
       [-0.97972275, -0.75239882, -0.56761993, ..., -1.04637821,
        -1.02958859, -1.02965915],
       ...,
       [-0.89489341, -0.80357724, -0.548873  , ..., -1.02367722,
        -1.01988873, -1.01541468],
       [-0.19688668, -0.4953505 , -0.55590951, ..., -0.66925112,
        -0.71627933, -0.75112044],
       [-0.97629715, -0.75338654, -0.35985653, ..., -1.06329517,
        -1.04739577, -1.05616564]])

In [0]:
BaggingClassifier?

In [0]:
# Define a Ada-boost classification model utilizing each of the base classifier used above

adaBoost_clfs = [rf, et, dtc, svc]

for clf in adaBoost_clfs[:-1]:
  # print(clf)
  adaboost_clf = AdaBoostClassifier(base_estimator=clf,
                                    n_estimators=100,
                                    random_state=0)
  adaboost_clf.fit(X_train, Y_train)
  adaboost_scores = adaboost_clf.predict_proba(X_test)
  print("[{} Ada-Boost] Score: {}".format(clf.__class__.__name__,
                                        calc_crps(Y_test_one_hot, pred_probs_to_199_classes(adaboost_scores, bagging_clf.classes_))))



[RandomForestClassifier Ada-Boost] Score: 0.016531
[ExtraTreesClassifier Ada-Boost] Score: 0.014774
[DecisionTreeClassifier Ada-Boost] Score: 0.025848


In [0]:
# Define our neural network as part of the ensemble (LATER)

# labels = ['DecisionTree', 'KNeighbors', 'Ridge Classifier', 'SVC']
# ensemble_clf = VotingClassifier([('Decision Tree', dtc), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)],
#                                 voting='soft')
# ensemble_clf = ensemble_clf.fit(X_train, Y_train)
# voting_classes = ensemble_clf.predict_proba(X_test)
# print("[{} Bagging] Score: {}".format(clf.__class__.__name__,
#                                         calc_crps(Y_test_one_hot,
#                                                   pred_probs_to_199_classes(voting_scores, voting_scores.classes_))))


# Define Voting-based regression model using the regression models we have previously used
# in phase #1.

# Define base regressors to use

vr = VotingRegressor()


In [0]:
# Define bagging-based regression models